In [ ]:
import pandas as pd
import os

# Dictionary for full state names to abbreviations
state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH',
    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Directory where your CSV files are stored
data_directory = "./States_perciptation"  # Replace with your folder path

# List to hold results
results = []

# Loop through all CSV files in the directory
for filename in os.listdir(data_directory):
    if filename.endswith(".csv"):  # Ensure it's a CSV file
        # Construct full file path
        file_path = os.path.join(data_directory, filename)
        
        # Read the CSV file and extract metadata lines (assuming metadata starts with #)
        with open(file_path, 'r') as file:
            lines = file.readlines()
        
        # Extract the state from the metadata (look for the first line starting with "#")
        state_name = None
        for line in lines:
            if line.startswith('#'):  # Look for metadata lines
                parts = line.split()  # Split by spaces
                if len(parts) > 2:
                    full_state_name = " ".join(parts[1:-1])  # Combine everything except the first and last parts
                    state_name = state_abbreviations.get(full_state_name, None)  # Map to abbreviation
                    if state_name:
                        break
        
        # If the state abbreviation was not found in the metadata, skip this file
        if state_name is None:
            print(f"State name not found in {filename}. Skipping file.")
            continue
        print(f"Processing state: {state_name}")
        
        # Read the actual data into a DataFrame (skip metadata lines)
        df = pd.read_csv(file_path, skiprows=4)  # Skip first 4 metadata lines
        
        # Ensure that 'Date' is in the correct datetime format
        df['Date'] = pd.to_datetime(df['Date'], format='%Y%m')
        
        # Extract the year from the 'Date' column
        df['Year'] = df['Date'].dt.year
        
        # Filter out the year 2020
        df = df[df['Year'] != 2020]
        
        # Group by 'Year' and calculate the average precipitation
        annual_avg = df.groupby('Year')['Value'].mean().reset_index()
        
        # Add state abbreviation column
        annual_avg['State'] = state_name
        
        # Append the result to the results list
        results.append(annual_avg)

# Concatenate all the results into a single DataFrame
final_df2 = pd.concat(results, ignore_index=True)
final_df2.to_csv('final_df2.csv', index=False)
